In [1]:
import numpy as np
import pandas as pd

In [2]:
all_landmarks = pd.read_csv('/home/sai/Downloads/FUNCTIEKAART.csv', sep=';')
super_markets = all_landmarks[all_landmarks.FUNCTIE2_ID == 'D03']
super_markets['lat'] = pd.Series([x.replace(',', '.') for x in super_markets.LAT.values], index=super_markets.index)
super_markets['lon'] = pd.Series([x.replace(',', '.') for x in super_markets.LNG.values], index=super_markets.index)
super_markets.lat = super_markets.lat.astype('float64')
super_markets.lon = super_markets.lon.astype('float64')
super_markets = super_markets.copy()

/home/sai/code/personal/py-snippets/.env/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/sai/code/personal/py-snippets/.env/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/sai/code/personal/py-snippets/.env/lib/python3.5/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be s

In [3]:
all_results = pd.read_pickle('all_panaroma_images.pkl')

In [4]:
one_img_per_sm = []
for i in range(super_markets.shape[0]):
    print(i)
    one_sm = super_markets.iloc[[i],:]
    tmp = all_results.copy()
    tmp['dist'] = pd.Series(
        np.sqrt(np.add(
    np.square(all_results.lat.values - one_sm.lon.values[0]),
    np.square(all_results.lon.values - one_sm.lat.values[0])
)),
                            index=tmp.index)
    tmp = tmp.sort_values('dist').head(1)
    tmp['OBJECTNUMMER'] = one_sm.OBJECTNUMMER.values[0]
    one_img_per_sm.append(tmp.copy())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163


In [5]:
one_img_per_sm = pd.concat(one_img_per_sm)

In [6]:
one_img_per_sm.head()

,href,lat,lon,timestamp,dist,OBJECTNUMMER
117,https://api.data.amsterdam.nl/panorama/recente...,4.923457,52.350692,2018-05-15T11:16:28.264650Z,0.000339,2222
14,https://api.data.amsterdam.nl/panorama/recente...,4.863935,52.363284,2017-04-21T13:12:54.613410Z,0.000193,3956
44,https://api.data.amsterdam.nl/panorama/recente...,4.793218,52.356386,2017-05-02T12:29:02.976370Z,0.000203,4033
977,https://api.data.amsterdam.nl/panorama/recente...,4.821896,52.381262,2018-01-08T12:19:11.905220Z,0.000448,4043
947,https://api.data.amsterdam.nl/panorama/recente...,4.874668,52.363507,2017-04-26T09:30:58.264610Z,0.000340,4052


In [9]:
one_img_per_sm.to_pickle('one_img_per_sm.pkl')

In [10]:
from PIL import Image
import urllib

def split_cubic(filepath, height=256, width=256):
    filepath = json.loads(urllib.request.urlopen(filepath).read().decode('utf-8'))['image_sets']['cubic']['preview']
    fd = urllib.request.urlopen(filepath)
    img = Image.open(fd)
    img_width, img_height = img.size
    image_array = []
    for i in range(0, img_height, height):
        for j in range(0, img_width, width):
            box = (j, i, j+width, i+height)
            crop_img = img.crop(box)
            image_array.append(crop_img)
    return image_array

In [11]:
import json

In [12]:
for i in range(one_img_per_sm.shape[0]):
    print(i)
    imgs = split_cubic(one_img_per_sm.href.values[i])
    imgs = [x for i,x in enumerate(imgs) if i not in [1,5]]
    for j in range(len(imgs)):
        imgs[j].save(str(i) + '-' + str(j) + '.jpg',
                     "JPEG", quality=100)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
